# FIEK Examination Timetabling using Linear Programming

In [1]:
# Libraries
import pulp 
import pandas as pd
import time

In [2]:
# DataFrame options
pd.set_option('display.max_rows',100)
pd.set_option('display.colheader_justify', 'center')

## Preparing the data for the model

In [3]:
# Reading the data from the excel

filename = "Shtator1-2019-20"

AllData = pd.ExcelFile("Datasets/" + filename + ".xlsx")
StudentData = pd.read_excel(AllData, 'StudentList')
TeacherData = pd.read_excel(AllData, 'TeacherList')
CourseData = pd.read_excel(AllData, 'CourseList')
RoomData = pd.read_excel(AllData, 'RoomList')
DayData = pd.read_excel(AllData, 'Ditet')
SlotData = pd.read_excel(AllData, 'Slots')
AllDataset = pd.read_excel(AllData, 'AllData')

In [4]:
# Defining the lenght and range of the data

num_students = len(StudentData)
num_teachers = len(TeacherData)
num_courses = len(CourseData)
num_rooms = len(RoomData)
num_days = len(DayData)
num_slots = len(SlotData)
AllDatasetLen = len(AllDataset)

all_students = range(num_students)
all_teachers = range(num_teachers)
all_courses = range(num_courses)
all_rooms = range(num_rooms)
all_days = range(num_days)
all_slots = range(num_slots)
AllDatasetRange = range(AllDatasetLen)

In [5]:
# Defining the entry data for the model

StudentList=StudentData["Studenti"]
TeacherList=TeacherData["Profesori"]

CourseList=CourseData["Lenda"]
SplitRoom=CourseData["SplitRoom"]
CourseEnrolled=CourseData["Enrolled"]
CourseCurricula=CourseData["Curricula"]
CurriculaType = set(CourseCurricula)

RoomList=RoomData["Room"]
CapacityRoom=RoomData["Capacity"]

Days=DayData['No.']
Slots=SlotData['No.']

# M dataset
StudentList2=AllDataset["Studenti"]
CourseList2=AllDataset["Lenda"]
TeacherList2=AllDataset["Profesori"]

In [6]:
# Dict for the Capacity room
a = list(CapacityRoom)
Capacity = {}
for key in RoomList:
    for value in a:
        Capacity[key] = value
        a.remove(value)
        break

# Dict for the course Enrollment
b = list(CourseEnrolled)
Enrolled = {}
for key in CourseList:
    for value in b:
        Enrolled[key] = value
        b.remove(value)
        break
        
# Dict for the course split
c = list(SplitRoom)
Split = {}
for key in CourseList:
    for value in c:
        Split[key] = value
        c.remove(value)
        break
        
# Dict for the course curricula
d = list(CourseCurricula)
Curricula = {}
for key in CourseList:
    for value in d:
        Curricula[key] = value
        d.remove(value)
        break

In [7]:
# Finding the minimum values
minvalues2 = []
minvalue2 = 0
for c in CourseList:
    if (Split[c] == 2):
        minvalues2.append(Enrolled[c])
if(len(minvalues2) != 0):
    minvalue2 = min(minvalues2)

minvalues3 = []
minvalue3 = 0
for c in CourseList:
    if (Split[c] == 3):
        minvalues3.append(Enrolled[c])
if(len(minvalues3) != 0):
    minvalue3 = min(minvalues3)

In [8]:
# Room combinations for two rooms
RoomCombinations = []
for c in CourseList:
    if(Split[c] == 2):
        for i in all_rooms:
            for j in all_rooms:
                if(RoomList[i] != RoomList[j]
                  and Capacity[RoomList[i]] > minvalue2
                  and Capacity[RoomList[j]] > minvalue2):
                    RoomCombinations.append((RoomList[i],RoomList[j]))
        break

# Room combinations for three rooms
RoomCombinations3 = []
for c in CourseList:
    if(Split[c] == 3):
        for i in all_rooms:
            for j in all_rooms:
                for k in all_rooms:
                    if(RoomList[i] != RoomList[j]
                      and RoomList[i] != RoomList[k] 
                      and RoomList[j] != RoomList[k]
                      and Capacity[RoomList[i]] > minvalue3
                      and Capacity[RoomList[j]] > minvalue3
                      and Capacity[RoomList[k]] > minvalue3):
                        RoomCombinations3.append((RoomList[i],RoomList[j],RoomList[k]))
        break

## The model

### Sets
$$  
 N = Student \; Set \;\;\;\; n \in N  \\
 T = Teacher \; Set \;\;\;\; t \in T \\
 C = Course \; Set \;\;\;\; c \in C \\
 R = Room \; Set \;\;\;\; r \in R \\
 D = Days  \;\;\;\; d \in D \\
 S = Slots \;\;\;\; s \in S \\
 P = Curricula \;\;\;\; p \in P \\
 K_{i} = Capacity  \\
 E_{i} = Enrollment \\
 Fi = Splitroom \\
 Pi = CourseCurricula \\
 M = Set \; of \; course \; for \; students \; for \; teacher \\
 $$

In [9]:
# Model defination
model = pulp.LpProblem('ExaminationTimetabling',sense=pulp.LpMinimize)

In [10]:
start_time = time.time()

#### Decisive Variable 1


\begin{equation*}
X_{nrsctp} = \begin{cases}
1 & \forall n, c, t \in M, \;\; \forall s, r, \;\; \forall Pi \in P, \;\;  \forall K_{i} >= E_{i}, \;\; \forall F_{i} = 1\\
0 & 
\end{cases}
\end{equation*}


In [11]:
# Decisive variable for each combinationes of courses that will not be splitted
X = {
        (CourseList2[i],StudentList2[i],r,d,s,TeacherList2[i],Curricula[CourseList2[i]]):pulp.LpVariable(f'{CourseList2[i]}   {r}  {d} {s} {StudentList2[i]} {TeacherList2[i]}  {Curricula[CourseList2[i]]}',cat=pulp.LpBinary)
        for r in RoomList
        for d in Days
        for s in Slots
        for i in AllDatasetRange
        if(Capacity[r] >= Enrolled[CourseList2[i]]
          and Split[CourseList2[i]] == 1)
}

#### Decisive Variable 2


\begin{equation*}
Y_{nrsctp} = \begin{cases}
1 & \forall n, c, t \in M, \;\; \forall s, r, \;\; \forall P_{i} \in P, \;\; \forall K_{i} >= E_{i}, \;\; \forall F_{i} = 2\\
0 & 
\end{cases}
\end{equation*}


In [12]:
# Decisive variable for each combinationes of courses that will be splitted
Y = {
        (CourseList2[i],StudentList2[i],r,d,s,TeacherList2[i],Curricula[CourseList2[i]]):pulp.LpVariable(f'{CourseList2[i]}   {r}  {d} {s}  {StudentList2[i]}  {TeacherList2[i]}  {Curricula[CourseList2[i]]}',cat=pulp.LpBinary)
        #for s in Slots
        for d in Days
        for s in Slots
        for i in AllDatasetRange 
        for r in RoomCombinations   
        if(Capacity[r[0]] >= Enrolled[CourseList2[i]] 
           and Capacity[r[1]] >= Enrolled[CourseList2[i]]  
           and Split[CourseList2[i]] == 2)
        
} 

#### Decisive Variable 3


\begin{equation*}
Z_{nrsctp} = \begin{cases}
1 & \forall n, c, t \in M, \;\; \forall s, r, \;\; \forall Pi \in P, \;\; \forall K_{i} >= E_{i}, \;\; \forall F_{i} = 3\\
0 & 
\end{cases}
\end{equation*}


In [13]:
# Decisive variable for each combinationes of courses that will be splitted
Z = {
        (CourseList2[i],StudentList2[i],r,d,s,TeacherList2[i],Curricula[CourseList2[i]]):pulp.LpVariable(f'{CourseList2[i]}   {r}  {d} {s}  {StudentList2[i]}  {TeacherList2[i]}  {Curricula[CourseList2[i]]}',cat=pulp.LpBinary)
        for d in Days
        for s in Slots
        for i in AllDatasetRange 
        for r in RoomCombinations3   
        if(Capacity[r[0]] >= Enrolled[CourseList2[i]] 
           and Capacity[r[1]] >= Enrolled[CourseList2[i]]
           and Capacity[r[2]] >= Enrolled[CourseList2[i]]
           and Split[CourseList2[i]] == 3)
        
}

In [14]:
Xdf = pd.DataFrame(X.items())
Ydf = pd.DataFrame(Y.items())
Zdf = pd.DataFrame(Z.items())
print(len(Xdf))
print(len(Ydf))
print(len(Zdf))
print('Our possible combinationes: ',len(Xdf) + len(Ydf) + len(Zdf))

95880
148320
409440
Our possible combinationes:  653640


#### Objective Function

$$ min \sum_{n \in M}^{} \sum_{c \in M}^{} \sum_{t \in M}^{N} \sum_{r}^{R} \sum_{s}^{S} \sum_{p}^{P} (X_{nrsctp} + Y_{nrsctp} + Z_{nrsctp}) \;\;\; $$

In [15]:
model += pulp.lpSum([v1 for k1, v1 in X.items()] + [v2 for k2, v2 in Y.items()] + [v3 for k3, v3 in Z.items()])

### Constraints

######  Constrain 1

$$ \sum_{p}^{P} \sum_{d}^{D} \sum_{s}^{S} (X_{nrdsctp} + Y_{nrdsctp} + Z_{nrsctp}) \le 1  \;\;\;\;\;\;\;\;\; \forall r \; \forall n, c, t \in M $$ 

In [16]:
# A course with the same curricula do not assign in the same day and timeslot
for p in CurriculaType:
    for d in Days:
        for s in Slots:
            model.addConstraint(sum([value for key, value in X.items() if key[3] == d and key[4] == s and key[6]==p]) 
                          + sum([value for key, value in Y.items() if key[3] == d and key[4] == s and key[6]==p])
                          + sum([value for key, value in Z.items() if key[3] == d and key[4] == s and key[6]==p])
                      <= 1)

######  Constrain 2

$$ \sum_{p}^{P} \sum_{d}^{D} (X_{nrdsctp} + Y_{nrdsctp} + Z_{nrsctp}) \le 1  \;\;\;\;\;\;\;\;\; \forall r,s, \; \forall n, c, t \in M $$ 

In [17]:
# A course with the same curricula do not assign in the same day and timeslot
for p in CurriculaType:
    for d in Days:
        model.addConstraint(sum([value for key, value in X.items() if key[3] == d and key[6]==p]) 
                      + sum([value for key, value in Y.items() if key[3] == d and key[6]==p])
                      + sum([value for key, value in Z.items() if key[3] == d and key[6]==p])
                  <= 1)


######  Constrain 3

$$ \sum_{d}^{D} \sum_{n}^{N} (X_{nrdsctp} + Y_{nrdsctp} + Z_{nrsctp}) \le 1  \;\;\;\;\;\;\;\;\; \forall r, s, p, \; \forall c, t \in M $$ 

In [18]:
# A student can not take more than one exam in a day
for d in Days:
    for n in StudentList:
        model.addConstraint(sum([value for key, value in X.items() if key[3] == d and key[1] == n]) 
                              + sum([value for key, value in Y.items() if key[3] == d and key[1] == n]) 
                              + sum([value for key, value in Z.items() if key[3] == d and key[1] == n])
                              <= 1)

######  Constrain 4

$$ \sum_{d}^{D} \sum_{s}^{S} \sum_{n}^{N} (X_{nrdsctp} + Y_{nrdsctp} + Z_{nrsctp}) \le 1  \;\;\;\;\;\;\;\;\; \forall r, p, \; \forall c, t \in M $$ 

In [19]:
# A student can not take more than one exam in a day and timeslot
for d in Days:
    for s in Slots:
        for n in StudentList:
            model.addConstraint(sum([value for key, value in X.items() if key[3] == d and key[4] == s and key[1] == n]) 
                                  + sum([value for key, value in Y.items() if key[3] == d and key[4] == s and key[1] == n])
                                  + sum([value for key, value in Z.items() if key[3] == d and key[4] == s and key[1] == n])
                                  <= 1)

######  Constrain 5

$$ \sum_{d}^{D} \sum_{t}^{T} (X_{nrdsctp} + Y_{nrdsctp} + Z_{nrsctp}) \le 1  \;\;\;\;\;\;\;\;\; \forall r, s, p, \; \forall c, n \in M $$ 

In [20]:
# A teacher can not hold more than one exam in a day
for d in Days:
    for t in TeacherList:
        model.addConstraint(sum([value for key, value in X.items() if key[3] == d and key[5] == t]) 
                              + sum([value for key, value in Y.items() if key[3] == d and key[5] == t])
                              + sum([value for key, value in Z.items() if key[3] == d and key[5] == t])
                              <= 1)

######  Constrain 6

$$ \sum_{d}^{D} \sum_{s}^{S} \sum_{t}^{T} (X_{nrdsctp} + Y_{nrdsctp} + Z_{nrsctp}) \le 1  \;\;\;\;\;\;\;\;\; \forall r, p \; \forall c, n \in M $$ 

In [21]:
# A teacher can not hold more than one exam in a day and timeslot
for d in Days:
    for s in Slots:
        for t in TeacherList:
            model.addConstraint(sum([value for key, value in X.items() if key[3] == d and key[4] == s and key[5] == t]) 
                                  + sum([value for key, value in Y.items() if key[3] == d and key[4] == s and key[5] == t])
                                  + sum([value for key, value in Z.items() if key[3] == d and key[4] == s and key[5] == t])
                                  <= 1)

##### Constrain 7
$$ \sum_{r}^{R} \sum_{c}^{C} ( X_{nrsct} + Y_{nrsct} + Z_{nrsctp})\le 1 \;\;\;\;\;\;\;\;\; \forall s, p, \;\; \forall n, t \in M $$

In [22]:
# A room can not be assigned for more than one exam
for r in RoomList:
    for c in CourseList:
        if(Capacity[r] >= Enrolled[c] and Split[c] == 1):
            model.addConstraint(sum([value for key, value in X.items() if key[2] == r  and key[0] == c]) <= 1 )

In [23]:
# A room can not be assigned for more than one exam
for r in RoomList:
    for c in CourseList:
        if(Capacity[r] >= Enrolled[c] and Split[c] == 2):
            model.addConstraint(sum([value for key, value in Y.items() if key[2][0] == r  and key[0] == c])
                                 + sum([value for key, value in Y.items() if key[2][1] == r  and key[0] == c]) <= 1 )

In [24]:
# A room can not be assigned for more than one exam
for r in RoomList:
    for c in CourseList:
        if(Capacity[r] >= Enrolled[c] and Split[c] == 3):
            model.addConstraint(sum([value for key, value in Z.items() if key[2][0] == r  and key[0] == c])
                                 + sum([value for key, value in Z.items() if key[2][1] == r  and key[0] == c])
                                 + sum([value for key, value in Z.items() if key[2][2] == r  and key[0] == c]) <= 1 )

##### Constrain 8

$$  \sum_{d}^{D} \sum_{s}^{S} \sum_{r}^{R} (X_{nrdsctp} + Y_{nrdsctp} + Z_{nrsctp}) \le 1 \;\;\;\;\;\;\;\;\; \forall p, \;\; \forall c, n, t \in M $$

In [25]:
# A room can not be assigned for more than one exam during a day
for d in Days:
    for r in RoomList:
        model.addConstraint(sum([value for key, value in X.items() if key[3] == d and key[2] == r]) 
                              + sum([value for key, value in Y.items() if key[3] == d and key[2][0] == r])
                              + sum([value for key, value in Y.items() if key[3] == d and key[2][1] == r])
                              + sum([value for key, value in Z.items() if key[3] == d and key[2][0] == r])
                              + sum([value for key, value in Z.items() if key[3] == d and key[2][1] == r])
                              + sum([value for key, value in Z.items() if key[3] == d and key[2][2] == r])
                              <= 1)

##### Constrain 9

$$  \sum_{c}^{C} \sum_{d}^{D} (X_{nrdsctp} + Y_{nrdsctp} + Z_{nrsctp}) \le 1 \;\;\;\;\;\;\;\;\; \forall n, t \in M,  \;\; \forall r, s, p  $$

In [26]:
# A exam can not be assigne for more than one time a day
for c in all_courses:
    for d in Days:
        model.addConstraint(sum([value for key, value in X.items() if key[3] == d and key[0] == CourseList[c]])
                              + sum([value for key, value in Y.items() if key[3] == d and key[0] == CourseList[c]])
                              + sum([value for key, value in Z.items() if key[3] == d and key[0] == CourseList[c]])
                              <= 1)

##### Constrain 10

$$  \sum_{c}^{C}  X_{nrdsctp} = 1  \;\;\;\;\;\;\;\;\; \forall F_{i} = 1, \;\; \forall n, t \in M,  \;\; \forall r, d, s, p $$

##### Constrain 11

$$  \sum_{c}^{C}  Y_{nrdsctp} = 1  \;\;\;\;\;\;\;\;\; \forall F_{i} = 2, \;\; \forall n, t \in M,  \;\; \forall r, d, s, p $$

##### Constrain 12

$$  \sum_{c}^{C}  Z_{nrdsctp} = 1  \;\;\;\;\;\;\;\;\; \forall F_{i} = 3, \;\; \forall n, t \in M,  \;\; \forall r, d, s, p $$

In [27]:
# The exam is guaranteed to be assigned one time
for c in all_courses:
    if(SplitRoom[c] == 1):
        model.addConstraint(sum([value for key, value in X.items() if key[0] == CourseList[c]]) == 1)
    if(SplitRoom[c] == 2):
        model.addConstraint(sum([value for key, value in Y.items() if key[0] == CourseList[c]]) == 1)
    if(SplitRoom[c] == 3):
        model.addConstraint(sum([value for key, value in Z.items() if key[0] == CourseList[c]]) == 1)

### Solver

In [28]:
current_time = time.time() 
reading_time = current_time - start_time 

# Solving the problem
model.solve()
sol = []
for var in model.variables():
    if var.varValue:
        sol.append(var.name)
        
solving_time = time.time() - current_time     

In [29]:
solving_time = time.time() - current_time

print('Our problem is '+ pulp.LpStatus[model.status])
print('Our objectve function',pulp.value(model.objective))
print('Our program needed', round(reading_time,0), 'seconds to read the data and', 
      round(solving_time,0), 'seconds to determine the optimal solution')

Our problem is Optimal
Our objectve function 37.0
Our program needed 136222.0 seconds to read the data and 229.0 seconds to determine the optimal solution


## Preparing the output

In [30]:
res = list(sol.copy())
resdf = pd.DataFrame(res)

In [31]:
# Converting LpVariable to a better format of a Dataset
Xdf = pd.DataFrame(X.items())
Ydf = pd.DataFrame(Y.items())
Zdf = pd.DataFrame(Z.items())

a, b, c = True, True, True
for c in CourseList:
    if(Split[c] == 1):
        if(a):
            new_column1 = Xdf[1].append(resdf[0],ignore_index = True)
            Xdf = Xdf.join(new_column1.rename('Values'), how='right')
            a = False
    if(Split[c] == 2):
        if(b):
            new_column2 = Ydf[1].append(resdf[0],ignore_index = True)
            Ydf = Ydf.join(new_column2.rename('Values'), how='right')
            b = False
    if(Split[c] == 3):
        if(c):
            new_column3 = Zdf[1].append(resdf[0],ignore_index = True)
            Zdf = Zdf.join(new_column3.rename('Values'), how='right')
            c = False
    
Df = pd.concat([Xdf, Ydf, Zdf])

In [32]:
Df = Df.drop([1], axis = 1)
Df[0] = Df[0].fillna(0)
Df['Values'] = Df['Values'].astype(str)
Df = pd.concat(g for _, g in Df.groupby('Values') if len(g) > 2)
Df.drop(Df.loc[Df[0] == 0].index, inplace=True)
Df = Df.reset_index(drop = True)  
Df.rename(columns = {0:'Key'}, inplace = True)

In [33]:
new_tuple = tuple(Df['Key'])
clean_data = [[item for item in row] for row in new_tuple]
Df = pd.DataFrame(clean_data, columns=list('0123456'))
Df = Df.drop(['1'], axis = 1)
Df.rename(columns = {'0':'Lenda', '2':'Salla', '3':'Day No.','4':'Periudha','5':'Profesori','6':'Curricula'}, inplace = True)
#ResultsDf = ResultsDf.style.set_properties(**{'text-align': 'left'})

In [34]:
# Working with Dataset for a better output
FinalResults = Df.sort_values(by=['Day No.'])
FinalResults = FinalResults.reset_index(drop = True)

In [35]:
FinalResults

,Lenda,Salla,Day No.,Periudha,Profesori,Curricula
0,Arkitektura e Kompjuterëve,"(408, 411)",2,1,Qamil Kabashi,v2p
1,Sinjalet dhe sistemet,"(404, 401)",2,1,Vjosa Shatri,v2s
2,Shkathtësi Komunikuse,201,2,1,Blerim Rexha,v1s
3,Menaxhimi i Projekteve,404,3,1,Sevdie Alshiqi,v2p
4,Matematika I,"(401, 404)",4,1,Shqipe Lohaj,v1s
5,Ndërmarrësi,201,4,1,Sevdie Alshiqi,v3s
6,Rrjetet Kompjuterike,408,5,1,Blerim Rexha,v2p
7,Bazat e Elektroteknikës,404,6,1,Luan Ahma,v1s
8,Bazat e Inteligjencës Artificiale,201,6,1,Nysret Musliu,v3p
9,Sistemet Operative,401,6,1,Isak Shabani,v3s


In [36]:
# Defining slots and days
SlotData = pd.read_excel(AllData, 'Slots')
SlotValue = SlotData["Slot"]
s = list(SlotValue)
SlotsDict = {}
for key in Days:
    for value in s:
        SlotsDict[key] = str(value)
        s.remove(value)
        break

DayData = pd.read_excel(AllData, 'Ditet')
DayValue = DayData["Ditet"]
d = list(DayValue)
DaysDict = {}
for key in Days:
    for value in d:
        DaysDict[key] = value
        d.remove(value)
        break

In [37]:
FinalResults['Periudha'] = FinalResults['Periudha'].map(SlotsDict)
FinalResults['Dita'] = FinalResults['Day No.'].map(DaysDict)
FinalResults['Orari'] = FinalResults['Dita'] #+ " - " + FinalResults['Periudha']
FinalResults = FinalResults.drop(['Periudha','Dita'], axis = 1)
FinalResults = FinalResults.reindex(columns=['Lenda','Orari','Salla','Profesori'])
FinalResults = FinalResults.reset_index(drop=True)

In [38]:
FinalResults

,Lenda,Orari,Salla,Profesori
0,Arkitektura e Kompjuterëve,E martë 1,"(408, 411)",Qamil Kabashi
1,Sinjalet dhe sistemet,E martë 1,"(404, 401)",Vjosa Shatri
2,Shkathtësi Komunikuse,E martë 1,201,Blerim Rexha
3,Menaxhimi i Projekteve,E merkure 1,404,Sevdie Alshiqi
4,Matematika I,E enjte 1,"(401, 404)",Shqipe Lohaj
5,Ndërmarrësi,E enjte 1,201,Sevdie Alshiqi
6,Rrjetet Kompjuterike,E premte 1,408,Blerim Rexha
7,Bazat e Elektroteknikës,E hënë 2,404,Luan Ahma
8,Bazat e Inteligjencës Artificiale,E hënë 2,201,Nysret Musliu
9,Sistemet Operative,E hënë 2,401,Isak Shabani


In [39]:
FinalResults.to_excel("Results/Output-" + filename + ".xlsx")